In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.io import loadmat, savemat
from scipy import signal
from tqdm import tqdm
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
import torch
from torch import nn

In [5]:
device = 'cpu'

In [6]:
# function to online 
def shift_scale_sample_4d(data, copy_data=True, do_flip=True):
    P_SHOULDER_LEFT, P_SHOULDER_RIGHT = 11, 12
    P_HIP_LEFT, P_HIP_RIGHT = 23, 24
    K_HEIGHT_TO_WIDTH = 1
    
    if copy_data:
        data = data.copy()
    
    # 0) flip Y
    if do_flip:
        data[:, :, :, 1] *= -1

    # 1) shifting
    origin_shoulders = 0.5*(data[:, :, P_SHOULDER_LEFT:P_SHOULDER_LEFT+1, :2] + data[:, :, P_SHOULDER_RIGHT:P_SHOULDER_RIGHT+1, :2] )
    # origin_hips = 0.5*(data[:, :, P_HIP_LEFT:P_HIP_LEFT+1, :2] + data[:, :, P_HIP_RIGHT:P_HIP_RIGHT+1, :2] )
    data = data - origin_shoulders

    # 2) scaling
    kx = 1 / data[:, :, P_SHOULDER_LEFT:P_SHOULDER_LEFT+1, 0]
    origin_hips_new = 0.5*(data[:, :, P_HIP_LEFT:P_HIP_LEFT+1, :2] + data[:, :, P_HIP_RIGHT:P_HIP_RIGHT+1, :2] )
    # origin_shoulders_new = 0.5*(data[:, :, P_SHOULDER_LEFT:P_SHOULDER_LEFT+1, :2] + data[:, :, P_SHOULDER_RIGHT:P_SHOULDER_RIGHT+1, :2] )

    ky = K_HEIGHT_TO_WIDTH / origin_hips_new[:, :, :, 1]
    data[:, :, :, 0] *=  kx
    data[:, :, :, 1] *=  -ky
    
    return data

In [7]:
def align_tensor_by_se(tensor_data, se_list, s_e_new=[5, 65], filling='fl'):
    
    good_fillings = ['fl', 'first/last', 'roll']
    assert filling in good_fillings, f"Error! filling={filling} should be in {good_fillings}!"
    
    f_tensor = tensor_data.copy()
    samples, duration, signals = f_tensor.shape
    
    s_e_new = np.asarray(s_e_new).reshape(-1, 2)
    
    if s_e_new.shape[0] == 1:
        s_e_new = np.repeat(s_e_new, samples, axis=0)
    elif s_e_new.shape[0] == samples:
        pass
    else:
        assert 0, f"Error, s_e_new.shape={s_e_new.shape} in np.array from"\
                  f" should have 1 or {samples} size of the first dimension"\
                  f"for given 'tensor_data'[{tensor_data.shape}]"
    
    print(s_e_new.shape)
    se_array = np.asarray(se_list)
    t = np.arange(0, duration)
    t_tensor = np.repeat(t.reshape(1, -1), samples, axis=0)
    
    
    k_stretching =  (s_e_new [:,1:2] - s_e_new [:,0:1])/(se_array[:,1:2] - se_array[:,0:1])   
    t_tensor_  = (t_tensor -  se_array[:, 0:1])*k_stretching + s_e_new[:,0:1]
    
    
    # Skipping the null procedures
    mask = (np.round(s_e_new,0) == np.round(se_array,0)).prod(axis=1).astype(bool)
    # print(mask)
    t_tensor_[mask] = t_tensor[mask]
    print(f"{mask.sum()} procedures (from {mask.size}) were skipped!")
    
    t_tensor = t_tensor_
    
    interp_period = None
    if filling in ['fl', 'first/last']:
        interp_period = None
    elif filling in ['roll']:
        interp_period = duration
    else:
        print(f"Warning! activity for 'filling'={filling} is not defined! Default: interp_period=None!!")
        
    
    
    for k in range(signals):
        for i in range(samples):
            f = f_tensor[i, :, k]
            t = t_tensor[i, :]
            f_new = np.interp(np.arange(0, duration),t, f, period=interp_period)
            f_tensor[i, :, k] = f_new
            
    return f_tensor

In [7]:
# data_4d__ = shift_scale_sample_4d(data_4d, copy_data=True, do_flip=True)
# data_4d__.shape

### loading data

In [12]:
mt_file_clean = loadmat('data/ALIGNED_SER_4d_full.mat')

In [13]:
mt_file_clean.keys()

dict_keys(['__header__', '__version__', '__globals__', 'labels', 'data', 'starts', 'stops'])

In [14]:
labels_full = mt_file_clean['labels'][0]
labels_full

array([ 0,  1,  2, ..., 27, 42, 45], dtype=int64)

In [15]:
data_full = mt_file_clean['data']
data_full.shape

(7420, 120, 67, 3)

In [16]:
data_4d = data_full[::,::,::,0:2]
data_4d.shape

(7420, 120, 67, 2)

In [17]:
# # scaling
data_4d = shift_scale_sample_4d(data_4d, copy_data=True, do_flip=True)
data_4d.shape

(7420, 120, 67, 2)

In [18]:
starts, stops = mt_file_clean['starts'][0], mt_file_clean['stops'][0]
(starts, stops)

(array([20., 20., 20., ..., 20., 20., 20.]),
 array([76. , 63. , 76. , ..., 73. , 77.5, 59. ]))

### Dataset preparation

In [19]:
data_3d_full = data_4d.reshape(data_4d.shape[0], data_4d.shape[1], data_4d.shape[2]*data_4d.shape[3])
data_3d_full.shape

(7420, 120, 134)

(7420, 2)

In [22]:
# scaling
se = np.stack([starts,stops], axis=1)
print(se.shape)
data_3d_full = align_tensor_by_se(data_3d_full, se, s_e_new=[5, 65])

(7420, 2)
(7420, 2)
0 procedures (from 7420) were skipped!


In [23]:
data_3d_train, data_3d_test, labels_train, labels_test = train_test_split(data_3d_full, labels_full, test_size=0.1)

data_3d_train.shape

(6678, 120, 134)

In [24]:
from sklearn.preprocessing import StandardScaler

data_2d_train = data_3d_train.reshape(data_3d_train.shape[0], data_3d_train.shape[1]*data_3d_train.shape[2])
data_2d_test = data_3d_test.reshape(data_3d_test.shape[0], data_3d_test.shape[1]*data_3d_test.shape[2])

scaler = StandardScaler()
scaler.fit(data_2d_train)

data_2d_train = scaler.transform(data_2d_train)
data_2d_test = scaler.transform(data_2d_test)

data_3d_train = data_2d_train.reshape(data_3d_train.shape[0], data_3d_train.shape[1], data_3d_train.shape[2])
data_3d_test =  data_2d_test.reshape(data_3d_test.shape[0], data_3d_test.shape[1], data_3d_test.shape[2])

data_3d_train.shape

(6678, 120, 134)

In [25]:
from torch.utils.data import Dataset, DataLoader

class Dataset_LSTM_Single(Dataset):
    def __init__(self, data, labels):
        data_pairs_list = []
        for sig, label in zip(data, labels):
            data_pairs_list.append([sig, label])
            
        self.data_pairs_list = data_pairs_list
        self.data = data
        self.labels = labels
        self.uniq_labels = len(np.unique(self.labels))

    def __len__(self):
        return len(self.data_pairs_list)

    def __getitem__(self, idx):
#         gesture_seqence, label = self.data_pairs_list[idx]
#         gesture_seqence = torch.tensor(gesture_seqence, dtype=torch.float)
#         label_tensor = torch.tensor(label, dtype=torch.long)
        
        gesture_seqence, label = self.data_pairs_list[idx]
        # slicing
        start_idx = np.random.randint(low=0, high=gesture_seqence.shape[0]-20)
        gesture_seqence = gesture_seqence[start_idx: start_idx+20]
        if start_idx < 10 or start_idx > gesture_seqence.shape[0]-30:
            label = self.uniq_labels
        
        gesture_seqence = torch.tensor(gesture_seqence, dtype=torch.float)
        label_tensor = torch.tensor(label, dtype=torch.long)
        
        
        
        return gesture_seqence, label_tensor

In [26]:
np.random.randint(low=0, high=data_3d_train[0].shape[0]-20)

53

In [27]:
DS_TRAIN = Dataset_LSTM_Single(data_3d_train, labels_train)
DL_TRAIN = DataLoader(DS_TRAIN, batch_size=4, shuffle=False)

DS_TEST = Dataset_LSTM_Single(data_3d_test, labels_test)
DL_TEST = DataLoader(DS_TEST, batch_size=4, shuffle=False)

for x, y in DL_TRAIN:
    print(x.shape, y.shape)
    break

torch.Size([4, 20, 134]) torch.Size([4])


### LSTM

In [28]:
from misc.LSTM_simple_classifier import LSTMTagger

import torch

In [29]:
LSTM_model = LSTMTagger(N_input_features=134, hidden_dim=128, N_lstm_layers=2, dropout=0.1, # lstm
                 N_1d_filters=3, kernel_size=5, # 1d convolution
                 target_size=len(np.unique(labels_full))+1, use_all_lstm_layers=True, # last linear layer
                 )

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(LSTM_model.parameters(), lr=3e-2, momentum=0.9)

No batchnorm
dropout


In [30]:
for seq, tags in DL_TRAIN:
#     LSTM_model.zero_grad()
    seq, tags = seq.to(device), tags.to(device)
    pred_tags = LSTM_model(seq)
    break

print(pred_tags.shape)

torch.Size([4, 54])


In [31]:
from torchmetrics import Accuracy, AUROC, F1Score, Precision, Recall, AveragePrecision
from collections import Counter

accuracy = Accuracy(task="multiclass", num_classes=len(Counter(labels_full)) + 1  ).to(device)
# f1_scorer = F1Score(task="multiclass", num_classes=len(Counter(labels_full))).to(device)
# precision = Precision(task="multiclass", num_classes=len(Counter(labels_full))).to(device)
# recall = Recall(task="multiclass", num_classes=len(Counter(labels_full))).to(device)

def validate(model, dl_test, metric=accuracy):
    res = []
    with torch.no_grad():
        model.eval()
        for seq, tags in dl_test:
            seq, tags = seq.to(device), tags.to(device)
            pred_tags = model(seq)
            
            res.append(metric(torch.argmax(pred_tags, dim=1), tags).item())
    model.train(True)
    return np.mean(res)

In [32]:
len(np.unique(labels_full))

53

In [33]:
acc_test = []
acc_list = []
losses = []

for epoch in tqdm(range(100)):
    count = 0
    for seq, tags in DL_TRAIN:
        LSTM_model.zero_grad()
        seq, tags = seq.to(device), tags.to(device)
        pred_tags = LSTM_model(seq)
        
#         print('tags', tags.shape)
#         print('pred_tags', pred_tags.shape)
        

#         pred_tags = pred_tags.view(-1, pred_tags.shape[-1])

        # Cross entropy input size is (N, C) N-batchsize, C-num_classes
        # we can treat L(sequence length) as multiple of N
        tags = tags.view(-1)
        loss = loss_fn(pred_tags, tags)
        
        acc_list.append(accuracy(torch.argmax(pred_tags, dim=1), tags).item())
    
        if count % 500 == 0:
            acc_test.append(validate(LSTM_model, DL_TEST, accuracy))
        count += 1
        
        loss.backward()
        losses.append(loss.item())
        optimizer.step()

  0%|                                                                                          | 0/100 [01:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
loss

### Classifiers

In [ ]:
LDA = LinearDiscriminantAnalysis()
KNN = KNeighborsClassifier(n_neighbors=5)
RF = RandomForestClassifier(max_depth=10, n_estimators=50)
LR = LogisticRegression()

### 1. Right/wrong window classifier

In [ ]:
window_data_true = []
window_data_false = []

window_size = 20
steps = np.arange(0,101,5)

for strt in tqdm(steps):
    data_3d_slice = data_3d_train[::,::,::]
    data_2d = data_3d_slice[::,strt:strt+window_size,::].reshape(data_3d_slice.shape[0],
                                                                 window_size*data_3d_slice.shape[2])
    if strt >= 20 and strt <= 30:
        window_data_true.append(data_2d)
    else:
        window_data_false.append(data_2d)
        
print(len(window_data_false))
print(len(window_data_true))

In [ ]:
def rnd_indx(coef=3/18):
    return np.random.choice(window_data_true[0].shape[0], size=int(window_data_true[0].shape[0]*coef))

In [ ]:
window_data_array_true = np.concatenate(window_data_true)
window_data_array_false = np.concatenate([x[rnd_indx()] for x in window_data_false])
window_data_array = np.concatenate([window_data_array_true, window_data_array_false])
window_labels = [1]*window_data_array_true.shape[0] + [0]*window_data_array_false.shape[0]


window_data_array, window_labels = shuffle(window_data_array, window_labels, random_state=0)

print(window_data_array.shape)
print(len(window_labels))

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression())])

In [ ]:
# scaler = StandardScaler()
# data_2d_window = scaler.fit_transform(window_data_array.T).T

In [ ]:
cv_scores = cross_val_score(estimator=pipe, X=window_data_array, y=window_labels)
print(cv_scores)

In [ ]:
LR_window_clf = Pipeline([('scaler', StandardScaler()), ('LR', LR)])
LR_window_clf.fit(window_data_array, window_labels)

In [ ]:
with open('LR_window_clf.pkl', 'wb') as f:
    pickle.dump(LR_window_clf, f)

### 2. Gesture classifier

In [ ]:
gesture_labels = np.concatenate([labels_train for i in range(len(window_data_true))])
gesture_data = window_data_array_true

pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression())])

cv_scores = cross_val_score(estimator=pipe, X=gesture_data, y=gesture_labels)
print(cv_scores)

In [ ]:
LR_gesture_clf = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression())])
LR_gesture_clf.fit(gesture_data, gesture_labels)

In [ ]:
with open('LR_gesture_clf.pkl', 'wb') as f:
    pickle.dump(LR_gesture_clf, f)

### 3. Testing Window LDA + Gesture LDA

In [ ]:
accuracy_list = []

window_size = 20
steps = np.arange(0,101,1)

for strt in tqdm(steps):
    data_3d_slice = data_3d_test[::,::,::]
    data_2d = data_3d_slice[::,strt:strt+window_size,::].reshape(data_3d_slice.shape[0],
                                                                 window_size*data_3d_slice.shape[2])
    
    pred_labels = np.ones_like(labels_test)*-1
    pred_window_mask = LR_window_clf.predict(data_2d)
    if np.sum(pred_window_mask) > 0:
        data_2d_masked = data_2d[pred_window_mask==1]
        pred_labels_masked = LR_gesture_clf.predict(data_2d_masked)
        pred_labels[pred_window_mask==1] = pred_labels_masked
    
    acc = accuracy_score(labels_test, pred_labels)
    accuracy_list.append(acc)

In [ ]:
plt.figure(figsize=(6,3))
plt.title(f'LR window={window_size}')
plt.plot(steps*2 + window_size//2, accuracy_list, 'r')

plt.grid(True)
plt.ylim(0,1)
plt.xlim(0,110)
plt.show()

### Shifting window accuracy check

In [ ]:
# from sklearn.utils import shuffle
# labels = shuffle(labels, random_state=0)

In [ ]:
lda_scores_mean = []
lda_scores_std = []
steps = np.arange(0,41,5)
window_size = 20
for strt in tqdm(steps):
    data_3d_slice = data_3d[::,::2,::]
    data_2d = data_3d_slice[::,strt:strt+window_size,::].reshape(data_3d_slice.shape[0],window_size*data_3d_slice.shape[2])
    
    scaler = StandardScaler()
    data_2d = scaler.fit_transform(data_2d.T).T
    
    cv_scores = cross_val_score(estimator=LDA, X=data_2d, y=labels)
    lda_scores_mean.append(np.mean(cv_scores))
    lda_scores_std.append(np.std(cv_scores))    

In [ ]:
data_3d_slice.shape

In [ ]:
1/len(np.unique(labels))

In [ ]:
plt.figure(figsize=(6,3))
plt.title(f'LDA window={window_size}')
plt.plot(steps*2 + window_size//2, np.array(lda_scores_mean) + 3*np.array(lda_scores_std), 'g', alpha=0.3)
plt.plot(steps*2 + window_size//2, lda_scores_mean, 'r')
plt.plot(steps*2 + window_size//2, np.array(lda_scores_mean) - 3*np.array(lda_scores_std), 'g', alpha=0.3)

plt.grid(True)
plt.ylim(0,1)
plt.xlim(0,110)
plt.show()

### Consequitive model

In [ ]:
window_size = 20

data_3d_resh = np.concatenate([data_3d[::,20:40,::],
                               ])

print(data_3d_resh.shape)

data_2d_resh = data_3d_resh.reshape(data_3d_resh.shape[0],
                                    data_3d_resh.shape[1]*data_3d_resh.shape[2])
print(data_2d_resh.shape)

In [ ]:
scaler = StandardScaler()
data_2d_resh = scaler.fit_transform(data_2d_resh.T).T

In [ ]:
cv_scores = cross_val_score(estimator=LDA,
                            X=data_2d_resh,
                            y=np.concatenate([labels]*1),
                            )
print(cv_scores)

### Sequential prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(data_3d, labels, test_size=0.2, random_state=42)
print(X_train.shape)

In [ ]:
window_size = 20

X1 = X_train[::,0:window_size,::].reshape(X_train.shape[0],window_size*X_train.shape[2])
X2 = X_train[::,20:20+window_size,::].reshape(X_train.shape[0],window_size*X_train.shape[2])
X3 = X_train[::,40:40+window_size,::].reshape(X_train.shape[0],window_size*X_train.shape[2])
X4 = X_train[::,60:60+window_size,::].reshape(X_train.shape[0],window_size*X_train.shape[2])
X5 = X_train[::,80:80+window_size,::].reshape(X_train.shape[0],window_size*X_train.shape[2])

print(X_train1.shape)

In [ ]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda_model
from sklearn.ensemble import RandomForestClassifier as lda_model
LDA1, LDA2, LDA3, LDA4, LDA5 = lda_model(), lda_model(), lda_model(), lda_model(), lda_model()
LDA1.fit(X1, y_train)
LDA2.fit(X2, y_train)
LDA3.fit(X3, y_train)
LDA4.fit(X4, y_train)
LDA5.fit(X5, y_train)

In [ ]:
p1, p2, p3, p4, p5 = [], [], [], [], []
for X_slice in [X1, X2, X3, X4, X5]:
    for model, p_list in zip([LDA1,LDA2,LDA3,LDA4,LDA5], [p1,p2,p3,p4,p5]):
        p_list.append(model.predict_proba(X_slice))

In [ ]:
print(accuracy_score(y_train, np.argmax((p1[0] + p2[0] + p3[0] + p4[0] + p5[0])/5,  axis=1)))
print(accuracy_score(y_train, np.argmax((p1[1] + p2[1] + p3[1] + p4[1] + p5[1])/5,  axis=1)))
print(accuracy_score(y_train, np.argmax((p1[2] + p2[2] + p3[2] + p4[2] + p5[2])/5,  axis=1)))
print(accuracy_score(y_train, np.argmax((p1[3] + p2[3] + p3[3] + p4[3] + p5[3])/5,  axis=1)))
print(accuracy_score(y_train, np.argmax((p1[4] + p2[4] + p3[4] + p4[4] + p5[4])/5,  axis=1)))

In [ ]:
aaa = (p1[0] + p2[0] + p3[0] + p4[0] + p5[0])/5
bbb = (p1[4] + p2[4] + p3[4] + p4[4] + p5[4])/5

plt.figure(figsize=(6,2))
plt.plot(aaa[50], 'r')
plt.plot(bbb[50], 'g')
plt.show()

In [ ]:
cnt = 0
for i in range(aaa.shape[0]):
    if np.max(aaa[i]) > np.max(bbb[i]):
        cnt += 1
print(cnt)

In [ ]:
print(accuracy_score(y_train, np.argmax(np.array(p5)[4], axis=1)))
print(accuracy_score(y_train, np.argmax(np.mean(np.array(p5),axis=0), axis=1)))

In [ ]:
p1m, p2m, p3m, p4m, p5m = np.mean(p1,axis=0),np.mean(p2,axis=0),np.mean(p3,axis=0),np.mean(p4,axis=0), np.mean(p5,axis=0)
ppm = (p1m + p2m + p3m + p4m + p5m)/5
print(accuracy_score(y_train, np.argmax(ppm,  axis=1)))

In [ ]:
final_features = np.concatenate(p1+p2+p3+p4+p5, axis=1)
print(final_features.shape)

LDA_final = lda_model()
LDA_final.fit(final_features, y_train)

### prediction

In [ ]:
window_size = 20

X1t = X_test[::,0:window_size,::].reshape(X_test.shape[0],window_size*X_train.shape[2])
X2t = X_test[::,20:20+window_size,::].reshape(X_test.shape[0],window_size*X_train.shape[2])
X3t = X_test[::,20:20+window_size,::].reshape(X_test.shape[0],window_size*X_train.shape[2])
X4t = X_test[::,20:20+window_size,::].reshape(X_test.shape[0],window_size*X_train.shape[2])
X5t = X_test[::,20:20+window_size,::].reshape(X_test.shape[0],window_size*X_train.shape[2])

print(X1t.shape)

p1t, p2t, p3t, p4t, p5t = [], [], [], [], []
for X_slice in [X1t, X2t, X3t, X4t, X5t]:
    for model, p_list in zip([LDA1,LDA2,LDA3,LDA4,LDA5], [p1t,p2t,p3t,p4t,p5t]):
        p_list.append(model.predict_proba(X_slice))
        

In [ ]:
p1mt, p2mt, p3mt, p4mt, p5mt = np.mean(p1t,axis=0),np.mean(p2t,axis=0),np.mean(p3t,axis=0),np.mean(p4t,axis=0), np.mean(p5t,axis=0)
ppmt = (p1mt + p2mt + p3mt + p4mt + p5mt)/5
print(accuracy_score(y_test, np.argmax((p1t[0] + p2t[0] + p3t[0] + p4t[0] + p5t[0])/5,  axis=1)))
print(accuracy_score(y_test, np.argmax((p1t[1] + p2t[1] + p3t[1] + p4t[1] + p5t[1])/5,  axis=1)))
print(accuracy_score(y_test, np.argmax((p1t[2] + p2t[2] + p3t[2] + p4t[2] + p5t[2])/5,  axis=1)))
print(accuracy_score(y_test, np.argmax((p1t[3] + p2t[3] + p3t[3] + p4t[3] + p5t[3])/5,  axis=1)))
print(accuracy_score(y_test, np.argmax((p1t[4] + p2t[4] + p3t[4] + p4t[4] + p5t[4])/5,  axis=1)))

In [ ]:
p1, p2, p3, p4, p5 = [], [], [], [], []
for X_slice in [X1, X2, X3, X4, X5]:
    for model, p_list in zip([LDA1,LDA2,LDA3,LDA4,LDA5], [p1,p2,p3,p4,p5]):
        p_list.append(model.predict_log_proba(X_slice))

In [ ]:
NNN = 11
print(y_test[NNN])
print(np.argmax(LDA.predict_proba(X_test[NNN].reshape(1, -1))))

plt.figure(figsize=(6,2))
plt.plot(LDA.predict_proba(X_test[NNN].reshape(1, -1))[0])
plt.yscale('log')
plt.ylim(1e-32, 10)
plt.show()

In [ ]:
def generate_resp(x):
    p1 = LDA.predict_proba(x.reshape(1, -1))
    p2 = LDA.predict_proba(x.reshape(1, -1))    
    p3 = LDA.predict_proba(x.reshape(1, -1))    
    p4 = LDA.predict_proba(x.reshape(1, -1))    
    p5 = LDA.predict_proba(x.reshape(1, -1))    
    p_res = np.concatenate([p1,p2,p3,p4,p5])
#     print(p_res.shape)
    return np.mean(p_res,axis=0)

In [ ]:
%timeit generate_resp(data_2d_resh[10].reshape(-1,1))